<h1>Assignment 5: Ground Truth Spreadsheet Reader</h1>
<br></br>

This notebook reads the [OCR results spreadsheet](https://docs.google.com/spreadsheets/d/1SzoM9PkAnUyaXuTQb-l-6DXCSH3hsI33x-i8v3ir-3I/edit#gid=0) and generates a Counter dictionary.

Table of contents:
* [Read from CSV](#csv)
* [Read from Google Sheets](#sheets)

## Read from CSV

Download the [OCR results spreadsheet](https://docs.google.com/spreadsheets/d/1SzoM9PkAnUyaXuTQb-l-6DXCSH3hsI33x-i8v3ir-3I/edit#gid=0) as a CSV in the Google Sheets interface and make sure it's in the same directory as your part 1 notebook. Copy and paste the following code in your part 1 notebook to get the counter.

In [ ]:
import pandas as pd
from collections import Counter

df = pd.read_csv('results.csv')
df.head()

In [ ]:
# remove columns we don't need
df.drop(['Page #', 'Student'], axis=1, inplace=True)
df.head()

In [ ]:
# replace NaNs with zeros
df = df.fillna(0)
df.head()

In [ ]:
def readTruth(df):
    """copies pairs of columns (i.e., Name2 and Count2) into temp df"""
    counter = Counter()
    for i in range(0, len(df.columns), 2):
        df_temp = df.iloc[:,i:i+2]
        # iterates through temp df and stores counts
        for index, row in df_temp.iterrows():
            vals = row.values.tolist()
            vals[1] = float(vals[1])
            try:
                counter[vals[0].lower()] += vals[1] # updates the Counter dictionary
            except:
                pass
    return counter

In [ ]:
counter = readTruth(df)
counter

You can stop here if you just want to get the counter from the CSV––if you want to learn how to read dynamically from Google Sheets, keep reading.

## Read from Google Sheets<a class="anchor" id="sheets"></a>

The Sheets API allows your code to copy over everything from the live Google Spreadsheet. This is useful if the data changes over time; in our case, maybe some students haven't added their counts yet. Instead of downloading a new CSV every time there's an update, we can ues the Sheets API to grab everything from our spreadsheet just by running a cell.

### Setting Up

Using the Sheets API requires a little bit of set-up.

Kevin McLaughlin wrote [a good tutorial](https://socraticowl.com/post/integrate-google-sheets-and-jupyter-notebooks/) for using the Google Sheets API in a Jupyter Notebook. However, it's a little bit outdated and includes a rather long extra step––if you're following his instructions, you'll need to enable the <b>Google Sheets API</b>, not the Google Drive API. You can skip the step 'Create credentials to use the Google Drive API' and go right to 'Create Service Account credentials,' where you'll be instructed to create a Service Account.

Here's a summary of the steps:
* Create your Google Developer Credentials
    * Create a Google Developer Project in the [Google Developer Console](https://console.developers.google.com/project)
    * Enable the Google Sheets API in your new project
    * Create a service account to use the Google Sheets API
    * Download the credentials json and store with the notebook file
* Install the gspread wrapper for Python
    * `pip install gspread oauth2client sf2gspread`
    * [gspread docs](https://gspread.readthedocs.io/en/latest/)
* Share the Google sheet with your service account's `client_email` from your json file
* Read Google sheet data into Jupyter notebook
    * Import libraries
    * Connect to your service account
    * Import the data
    * Convert the data into a pandas dataframe
* Write to the Google Sheet

### Import gspread and Authorize Access

In [ ]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./credentials.json', scope)
gc = gspread.authorize(credentials)

### 'Open' the Spreadsheet and Read Data

In [ ]:
# the spreadsheet key comes from the URL of our spreadsheet
spreadsheet_key = '1SzoM9PkAnUyaXuTQb-l-6DXCSH3hsI33x-i8v3ir-3I'

book = gc.open_by_key(spreadsheet_key)
worksheet = book.worksheet("Sheet1")
table = worksheet.get_all_values()

# each row becomes a list, even the column headers
table[0:2]

### Convert Table to a DataFrame

In [ ]:
dfs = pd.DataFrame(table[1:], columns=table[0])

# remove the cols we don't need
dfs.drop(['Page #', 'Student'], axis=1, inplace=True)

# replace blanks with zeros
dfs.replace(r'^\s*$', 0, regex=True, inplace=True)

dfs.head()

### Use the Function From Earlier to Generate a Counter

In [ ]:
sheet_counter = readTruth(dfs)
sheet_counter

### Write the Totals Back to Our Sheet (in a new worksheet called 'Totals')

If we want to write this counter information to our Google Sheet, we can start by making a new DataFrame from our counter.

In [ ]:
df_totals = pd.DataFrame.from_dict(sheet_counter, orient='index')
df_totals.rename(columns={0: 'count'}, inplace=True)

df_totals

We set our worksheet name to 'Totals' and use another gspread package, df2gspread, which helps us upload a DataFrame to a spreadsheet.

In [ ]:
from df2gspread import df2gspread as d2g

wks_name = 'Totals'
d2g.upload(df_totals, spreadsheet_key, wks_name, credentials=credentials, row_names=True)